<a href="https://colab.research.google.com/github/AndreaHernandez5072/CienciaDatos/blob/main/HT9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Andrea Marleny Hernández Contreras, 1203718

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import PolynomialFeatures
#svm -> suport virtual machine
from sklearn.svm import SVC

In [3]:
df = pd.read_csv("orange.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   241 non-null    int64  
 1   Size         241 non-null    float64
 2   Weight       241 non-null    int64  
 3   Brix         241 non-null    float64
 4   pH           241 non-null    float64
 5   Softness     241 non-null    float64
 6   HarvestTime  241 non-null    int64  
 7   Ripeness     241 non-null    float64
 8   Color        241 non-null    object 
 9   Variety      241 non-null    object 
 10  Blemishes    241 non-null    object 
 11  Quality      241 non-null    float64
dtypes: float64(6), int64(3), object(3)
memory usage: 22.7+ KB


In [10]:
df.isna().sum()

Unnamed: 0     0
Size           0
Weight         0
Brix           0
pH             0
Softness       0
HarvestTime    0
Ripeness       0
Color          0
Variety        0
Blemishes      0
Quality        0
dtype: int64

In [7]:
print("Colors: ",df["Color"].unique())
print("\nVariety:",df["Variety"].unique())
print("\nBlemishes",df["Blemishes"].unique())

Colors:  ['Orange' 'Deep Orange' 'Light Orange' 'Orange-Red' 'Yellow-Orange']

Variety: ['Valencia' 'Navel' 'Cara Cara' 'Blood Orange' 'Hamlin' 'Tangelo (Hybrid)'
 'Murcott (Hybrid)' 'Moro (Blood)' 'Jaffa' 'Clementine' 'Washington Navel'
 'Star Ruby' 'Tangerine' 'Ambiance' 'California Valencia'
 'Honey Tangerine' 'Navel (Late Season)' 'Clementine (Seedless)' 'Temple'
 'Minneola (Hybrid)' 'Satsuma Mandarin' 'Midsweet (Hybrid)'
 'Navel (Early Season)' 'Ortanique (Hybrid)']

Blemishes ['N' 'Y (Minor)' 'Y (Sunburn)' 'Y (Mold Spot)' 'Y (Bruise)'
 'Y (Split Skin)' 'Y (Sunburn Patch)' 'Y (Scars)'
 'Y (Minor Insect Damage)' 'Y (Bruising)' 'N (Minor)' 'N (Split Skin)']
